# Required Libraries

In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.2 MB/s eta 

In [3]:
import os
import json
import torch
import chromadb
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from chromadb.config import Settings
from transformers import CLIPVisionModel, RobertaModel, AutoTokenizer, CLIPFeatureExtractor

# Download Data

In [4]:
!git clone https://github.com/FaSha20/Natural-Language-Processing-Projects

Cloning into 'Natural-Language-Processing-Projects'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 75 (delta 18), reused 38 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 8.07 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [5]:
with open("/content/Natural-Language-Processing-Projects/MultimodalRAG/scientists_data/scintists_data_with_text_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(len(data), "records loaded")

300 records loaded


# Build Embeddings and DATABASE

## Load Pre-trained Encoder Model

In [6]:
vision_encoder = CLIPVisionModel.from_pretrained('SajjadAyoubi/clip-fa-vision')
preprocessor = CLIPFeatureExtractor.from_pretrained('SajjadAyoubi/clip-fa-vision')
text_encoder = RobertaModel.from_pretrained('SajjadAyoubi/clip-fa-text')
tokenizer = AutoTokenizer.from_pretrained('SajjadAyoubi/clip-fa-text')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/350M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/354 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

## Encoding Functions

In [7]:
# ---- Text Encoder ----
def embed_text(text: str):
    text_embedding = text_encoder(**tokenizer(text, return_tensors='pt')).pooler_output
    return text_embedding[0]

# ---- Image Encoder ----
def embed_image(url: str):
    try:
        headers = {"User-Agent": "ImageDatasetScript/1.0 (email@gmail.com)"}
        response = requests.get(url, headers=headers, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_embedding = vision_encoder(**preprocessor(image, return_tensors='pt')).pooler_output
        return image_embedding[0]
    except Exception as e:
        return None

## Build the DATABASE

In [8]:
PERSIST_DIR = "/content/chroma_scientists_db"
os.makedirs(PERSIST_DIR, exist_ok=True)

client = chromadb.PersistentClient(path=PERSIST_DIR, settings=Settings(allow_reset=True))

text_db  = client.get_or_create_collection(name="scientists_text",  metadata={"hnsw:space": "cosine"})
image_db = client.get_or_create_collection(name="scientists_image", metadata={"hnsw:space": "cosine"})

In [9]:
def upsert_pair(idx, name, text, image_url, t_emb, i_emb):
    meta = {
        "pair_id": idx,
        "name": name,
        "text": text,
        "image_url": image_url,
    }

    if t_emb is not None:
        text_db.add(
            ids=[f"text-{idx}"],
            embeddings=[t_emb.tolist()],
            metadatas=[meta],
        )

    if i_emb is not None:
        image_db.add(
            ids=[f"img-{idx}"],
            embeddings=[i_emb.tolist()],
            metadatas=[meta],
        )

In [10]:
for idx, person in enumerate(data):
    name = person.get("name")

    text = person.get("text_chunk", "").replace("\u200c", "")
    t_emb = embed_text(text)

    image_urls = list(person.get("image", {}).values())
    for url in image_urls:
        if url:
            i_emb = embed_image(url)
            if i_emb is not None:
              img_url = url
              break

    upsert_pair(idx, name, text, img_url, t_emb, i_emb)

## Retrieval Functions

In [11]:
def search_by_text(query: str, k=3):
    q_emb = embed_text(query)
    res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

def search_by_image(image_url: str, k=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return None
    res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

## Test an example

In [17]:
res = search_by_text("فیلسوف ایرانی دوران ساسانی", k=3)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: اوستَن score: 0.7518327832221985
text: اوستَن، با نام‌های مستعار اوستانوس، هیوسایتانه، هیوستانیوس و استانس، مردی از دوران هخامنشی/هلنیستی بود. او به عنوان جادوگر، کیمیاگر و مغ شناخته می‌شد و آثاری همچون اکتاتک (نوشته‌ای در هشت کتاب) و متون جادوگری را به نگارش درآورد. از وقایع مهم زندگی او می‌توان به همراهی با خشایارشا در حمله به یونان در حدود ۱۲۰۰ سال پیش از هجرت و معرفی جادو به یونانیان اشاره کرد. همچنین، اوستن به عنوان آموزگار دموکریت، بر فلسفه یونانی نیز تأثیرگذار بود.
image: https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/ZodFarzad_Nazem.jpg/220px-ZodFarzad_Nazem.jpg 

name: نرسه score: 0.7515657544136047
text: نرسه، که با نام‌های نرسیِ شاپور و نرسی نیز شناخته می‌شود، از شاهنشاهان دوره ساسانیان بود. او در سال ۳۲۸ هجری شمسی با حمایت بزرگان و مخالفان بهرام سوم به سلطنت رسید، اما دوره حکومتش با جنگ‌هایی با رومیان همراه بود. شکست سنگین او از رومیان در نبرد ساتالا منجر به امضای عهدنامه تحقیرآمیز نصیبین و واگذاری سرزمین‌هایی به روم شد. این شکست‌ها در نهایت به استعفای نرسه